## AgentCore Runtime での動的クライアント登録

この例では、動的クライアント登録（Dynamic Client Registration）をサポートする MCP を AgentCore Runtime にデプロイする方法を示します。

このチュートリアルでは、この AgentCore 機能をサポートする Auth0 との統合例を作成します。

これは、例えばユーザーのソーシャルメディアログインを使用してユーザーを動的に登録し、MCP に接続できるようにする場合に便利です。

このチュートリアルでは、以下を学習します：

* ツールを持つ MCP サーバーの作成方法
* サーバーをローカルでテストする方法
* Auth0 テナントを DCR をサポートするように設定し、API とアプリを追加する方法
* Auth0 上の DCR と統合して、サーバーを AWS にデプロイする方法
* デプロイしたサーバーを呼び出す方法

### チュートリアル詳細

| 情報                | 詳細                                                      |
|:--------------------|:----------------------------------------------------------|
| チュートリアルタイプ | Auth0 でのツールホスティング + DCR                        |
| ツールタイプ         | MCP サーバー                                              |
| チュートリアルコンポーネント | AgentCore Runtime でのツールホスティング、MCP サーバーの作成 |
| チュートリアル分野   | クロスバーティカル                                        |
| 難易度              | 中級                                                      |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK および MCP Client      |

### チュートリアルアーキテクチャ

このチュートリアルでは、この例を AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、3つのツール（`add_numbers`、`multiply_numbers`、`greet_users`）を持つ非常にシンプルな MCP サーバーを使用します。

![Architecture](images/architecture.png)

### チュートリアルの主な機能

* MCP サーバーのホスティング
* 動的クライアント登録（DCR）
* Auth0

In [ ]:
!pip install -Uq -r requirements.txt

**インストールした依存関係を反映するためにカーネルを再起動してください。**

## Auth0 の設定

### 一般設定

この例では、Auth0 を使用して DCR（動的クライアント登録）を実装します。開始する前に、Auth0 コンソールで DCR オプションを有効にしましょう。

- 左パネルメニューで `Settings` をクリックします。次に **Tenant Settings** の下で `Advanced` オプションをクリックします。*Dynamic Client Registration (DCR)* オプションが見つかるまで下にスクロールし、クリックして *有効* にします。また、*Enable Application Connections* をクリックして、新しいアプリケーションが作成されたときに接続が有効になるようにします：

![Enable DCR](images/01_enable_dcr.png)

詳細については、Auth0 の[動的クライアント登録](https://auth0.com/docs/get-started/applications/dynamic-client-registration)ドキュメントを参照してください。

有効にしたので、MCP 上のユーザー（アプリ）のログインメカニズムとして *Google / Gmail* ID を使用しましょう。

- 左パネルメニューで `Authentication` をクリックし、次に `Social` をクリックします。**Social Connections** 画面で、`google-oauth2` オプションをクリックします。設定画面で、`Promote Connection to Domain Level` オプションが見つかるまで下にスクロールし、有効にします：

![Enable Social](images/02_enable_social.png)

これで Auth0 テナントのすべての設定が完了しました。次に進んで Auth0 API を作成しましょう。

---

### API の作成

API を作成しましょう。左パネルメニューの Applications セクションの下で `APIs` をクリックします。APIs 画面で、右上の `+ Create API` ボタンをクリックします。

![API](images/03_create_api.png)

- Name: API の名前。
- Identifier: API の一意の識別子。この値は認可呼び出しの `audience` パラメータとして使用されます。
- 他のオプションはデフォルトのままにして、Create をクリックできます。

API 設定で、`Permissions` タブをクリックし、次のパターンに従って invoke 権限を追加します `<identifier_name>:Invoke`：

![Permissions](images/04_permissions.png)

完了。これで API が設定されました。アプリを作成しましょう。

---

### アプリケーションの作成

アプリケーションを作成して設定を完了しましょう。左パネルメニューの `Applications` で `Applications` をクリックし、次に `+ Create Application` をクリックします。

アプリ作成画面で、アプリに名前を付け、`Machine-to-Machine` オプションを選択し、create をクリックします：

![App](images/05_app.png)

次の画面で、API を選択するよう求められます。前のステップで作成した API を選択し、`all` ボタンをクリックしてすべてのアクションを認可し、`Authorize` ボタンをクリックします：

![Auth](images/06_authorizing.png)

これでアプリが作成されました。

---

### アプリ情報の取得

この例で使用できるように、アプリ情報を取得する必要があります。

アプリをクリックして `quickstart` ページに移動します。このページには、必要なすべてのパラメータを含む Linux `curl` の例が表示されます。次の例のように、`--url` 値からドメインをコピーします：

`--url` パラメータ：

```bash
--url https://<your-auth0-tenant>.us.auth0.com/oauth/token
```

このノートブックで使用される次の環境変数を構築します：

```bash
export DISCOVERY_URL="https://<your-auth0-tenant>.us.auth0.com/.well-known/openid-configuration"
```

**ここにあなたのドメインを入力してください**

In [ ]:
DISCOVERY_URL="https://<your-auth0-tenant>.us.auth0.com/.well-known/openid-configuration"

audience も入力する必要があります。これは API 識別子になります。この例に従って `ac-runtime-api` を使用した場合、それが audience の値になります：

In [ ]:
# Replace if you created with a different identifier
AUDIENCE="ac-runtime-api"

### MCP サーバーの作成

それでは、3つのシンプルなツールを持つ MCP サーバーを作成しましょう：

In [ ]:
%%writefile server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """2つの数字を足し合わせる"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """2つの数字を掛け合わせる"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """ユーザーに名前で挨拶する"""
    return f"こんにちは、{name}さん！お会いできて嬉しいです。"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### MCP サーバーのローカルテスト（オプション）

必要であれば、新しく作成した MCP サーバーをローカルでテストできます。

次のセルを実行して、テスト用 Python スクリプトを生成します。

In [ ]:
%%writefile test_my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("利用可能なツール:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

MCP サーバーをローカルでテストするには：

1. **ターミナル 1**: MCP サーバーを起動

```bash
python server.py
```

2. **ターミナル 2**: テストスクリプトを実行

```bash
python test_my_mcp_client.py
```

### MCP サーバーを AgentCore Runtime に起動

提供された URL と audience を使用して AgentCore Runtime アプリを作成しましょう。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "mcp_dcr_sample"

auth_config = {
    "customJWTAuthorizer": {
        "allowedAudience": [
            AUDIENCE
        ],
        "discoveryUrl": DISCOVERY_URL,
    }
}

response = agentcore_runtime.configure(
    entrypoint="server.py",
    auto_create_execution_role=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration=auth_config,
    protocol="MCP",
    memory_mode="NO_MEMORY",
    deployment_type="direct_code_deploy",
    runtime_type="PYTHON_3_13",
)

In [ ]:
print("MCP サーバーを AgentCore Runtime に起動中...")
print("数分かかる場合があります...")
launch_result = agentcore_runtime.launch()
print("起動完了 ✓")
print(f"エージェント ARN: {launch_result.agent_arn}")
print(f"エージェント ID: {launch_result.agent_id}")

### テスト

ファイル `mcp_auth0_client.py` には、AgentCore Runtime 上にデプロイされたエージェントに接続するためのローカル実装が含まれています。

スクリプトを実行すると、ログインするために Google ID を求めるログインページにリダイレクトされます。

ログイン後、Auth0 があなたのメールを使用してアプリを作成することを認可するよう求められます。

![Redirect](images/07_redirect.png)

承認すると、成功を示すプレーンな HTML ページ（`mcp_auth0_client.py` で実装）にリダイレクトされます。

![OK](images/08_success.png)

スクリプトが待機している環境変数を設定

In [ ]:
agent_arn = launch_result.agent_arn
region = "us-west-2"
custom_endpoint =f"https://bedrock-agentcore.{region}.amazonaws.com"

agent_arn, custom_endpoint, AUDIENCE

In [ ]:
import mcp_auth0_client as mcp_client

await mcp_client.main(agent_arn, custom_endpoint, AUDIENCE)

## クリーンアップ

In [ ]:
agentcore_runtime.destroy()

### 🎉 おめでとうございます！

以下を正常に完了しました：

- Auth0 で**テナントを作成**
- Auth0 で **DCR を有効化**し、API とアプリを作成
- カスタムツールを持つ **MCP サーバー**を作成
- MCP クライアントを使用して**ローカルでテスト**
- **DCR による認証**をセットアップ
- **AgentCore Runtime** を使用して AWS にデプロイ
- 適切な認証で**リモートから呼び出し**
- MCP のコンセプトとベストプラクティスを**学習**

MCP サーバーは現在 Amazon Bedrock AgentCore Runtime 上で実行されており、本番環境で使用する準備ができています！